In [1]:
# 1. import libraries
import requests # to download html code
from bs4 import BeautifulSoup # to navigate through the html code
import pandas as pd
import numpy as np
import re

In [2]:
# We use the dataset from kaggle https://www.kaggle.com/datasets/saurabhshahane/music-dataset-1950-to-2019?resource=download
# containing over 28 k songs between 1950 and 2019

songs = pd.read_csv("C:/Users/e.sansebastian/OneDrive - EXPONDO/Ironhack\Weeks/Week_6/Day_1/Afternoon/lab-not-hot-songs/tcc_ceds_music.csv")
songs.head()

,Unnamed: 0,artist_name,track_name,release_date,genre,lyrics,len,dating,violence,world/life,...,sadness,feelings,danceability,loudness,acousticness,instrumentalness,valence,energy,topic,age
0,0,mukesh,mohabbat bhi jhoothi,1950,pop,hold time feel break feel untrue convince spea...,95,0.000598,0.063746,0.000598,...,0.380299,0.117175,0.357739,0.454119,0.997992,0.901822,0.339448,0.137110,sadness,1.0
1,4,frankie laine,i believe,1950,pop,believe drop rain fall grow believe darkest ni...,51,0.035537,0.096777,0.443435,...,0.001284,0.001284,0.331745,0.647540,0.954819,0.000002,0.325021,0.263240,world/life,1.0
2,6,johnnie ray,cry,1950,pop,sweetheart send letter goodbye secret feel bet...,24,0.002770,0.002770,0.002770,...,0.002770,0.225422,0.456298,0.585288,0.840361,0.000000,0.351814,0.139112,music,1.0
3,10,pérez prado,patricia,1950,pop,kiss lips want stroll charm mambo chacha merin...,54,0.048249,0.001548,0.001548,...,0.225889,0.001548,0.686992,0.744404,0.083935,0.199393,0.775350,0.743736,romantic,1.0
4,12,giorgos papadopoulos,apopse eida oneiro,1950,pop,till darling till matter know till dream live ...,48,0.001350,0.001350,0.417772,...,0.068800,0.001350,0.291671,0.646489,0.975904,0.000246,0.597073,0.394375,romantic,1.0


In [3]:
# We check the shape to see how many records we have
songs.shape

(28372, 31)

In [4]:
# We isolate only the columns "artist_name"and "track_name" and perform some changes in the columns to align them with the df hot_songs

not_hot = songs[["artist_name","track_name"]]
not_hot= not_hot.rename(columns={"artist_name": "artist", "track_name": "title"})
display(not_hot.head())
display(not_hot.shape)

,artist,title
0,mukesh,mohabbat bhi jhoothi
1,frankie laine,i believe
2,johnnie ray,cry
3,pérez prado,patricia
4,giorgos papadopoulos,apopse eida oneiro


(28372, 2)

In [5]:
# We pick a sample of only 3000 random songs to make the file a bit "lighter" faster to process
not_hot = not_hot.sample(n=3000, random_state=1)
not_hot.shape

(3000, 2)

In [6]:
# We load the hot_songs df we created in the previous lab:

hot_songs= pd.read_csv("C:/Users/e.sansebastian/OneDrive - EXPONDO/Ironhack/Weeks/Week_6/Day_1/Morning/lab-web-scraping-single-page/hot_100_billboard.csv")
hot_songs.head()

,artist,title
0,jack harlow,lovin on me
1,taylor swift,cruel summer
2,tate mcrae,greedy
3,doja cat,paint the town red
4,zach bryan featuring kacey musgraves,i remember everything


In [7]:
# We look for potential duplicates in both the hot_song df:

hot_songs.groupby("title").size().sort_values(ascending=False)

title
23                                   1
pretty little poison                 1
slime you out                        1
segun quien                          1
save me the trouble                  1
                                    ..
hey driver                           1
harley quinn                         1
greedy                               1
great gatsby                         1
you're losing me (from the vault)    1
Length: 100, dtype: int64

In [8]:
# We concatenate both not_hot and hot_songs df to check for duplicates before applying a function to remove them all
checking_df = pd.concat([not_hot, hot_songs], ignore_index=True)
checking_df.head()

,artist,title
0,glenn miller,the little man who wasn't there
1,misfits,american psycho
2,elliott smith,somebody that i used to know
3,june carter cash,juke box blues
4,"emerson, lake & palmer","karn evil 9 1st impression, pt. 1"


In [9]:
# We check the total shape of the concatenated df:
checking_df.shape

(3100, 2)

In [10]:
checking_df["artist_title"] = checking_df["artist"]+"_"+checking_df["title"]
checking_df.sort_values(by="title",ascending=False)

,artist,title,artist_title
240,rania,منك بخاف,rania_منك بخاف
386,tool,ænema,tool_ænema
1098,cultura profética,¿qué será? / ritmo que pesa / suelta los amarr...,cultura profética_¿qué será? / ritmo que pesa ...
352,kyle park,yours and mine,kyle park_yours and mine
138,rita ora,your song,rita ora_your song
...,...,...,...
1119,big daddy kane,'nuff respect,big daddy kane_'nuff respect
2857,louis armstrong,'bout time,louis armstrong_'bout time
2249,bleu edmondson,$50 dollars and a flask of crown,bleu edmondson_$50 dollars and a flask of crown
1136,garbage,#1 crush,garbage_#1 crush


In [36]:
# Before creating the function we check the occurrences of the songs on the concatenated table:

grouped_songs=pd.DataFrame(checking_df.groupby("artist_title").size().sort_values(ascending=False))
grouped_songs= grouped_songs.rename(columns={0: "occurrences"}).reset_index()
display(grouped_songs)
duplicated_songs= grouped_songs[grouped_songs["occurrences"] > 1]
duplicated_songs_sum = duplicated_songs["occurrences"].sum()
total_records = len(duplicated_songs["artist_title"])

print(f"Sum of duplicates of songs present in both df: {duplicated_songs_sum} from a total of {total_records} records")

,artist_title,occurrences
0,"""weird al"" yankovic_theme from rocky xiii",1
1,pet shop boys_always on my mind,1
2,phish_the mango song,1
3,phish_waste,1
4,phish_wilson,1
...,...,...
3095,george jones_a house without love,1
3096,george jones_it's a sin,1
3097,george jones_just when i needed you,1
3098,george jones_lonely christmas call,1


Sum of duplicates of songs present in both df: 0 from a total of 0 records


In [52]:
def remove_duplicates(df1, df2)-> pd.DataFrame:
    #converting all values of both dataframes to lower case
    df1 = df1.applymap(lambda x: x.lower() if isinstance(x, str) else x)
    df2= df2.applymap(lambda x: x.lower() if isinstance(x, str) else x)
    # performing a left merge to obtain the rows in the not hot songs that are present in the hot songs
    left_merge_df = df1.merge(df2, indicator=True, how='left')
    # keeping only the rows that are only present in the not hot dataframe
    result_df= left_merge_df.query('_merge == "left_only"').drop('_merge', axis=1)
    return result_df

In [53]:
not_hot_songs= remove_duplicates(not_hot,hot_songs)

In [56]:
display(not_hot_songs.shape)
not_hot_songs.head()

(3000, 2)

,artist,title
0,glenn miller,the little man who wasn't there
1,misfits,american psycho
2,elliott smith,somebody that i used to know
3,june carter cash,juke box blues
4,"emerson, lake & palmer","karn evil 9 1st impression, pt. 1"


In [57]:
not_hot_songs.to_csv('not_hot_songs.csv',index=False)